# Rekordbox-Find-Tempo-Changes-Inator

This notebook identifies tracks that have at least one tempo change.

## Import useful libraries

In [1]:
import pickle
import pandas as pd
import numpy as np

## User settings

In [2]:
output_data_directory = '../output_data'

## Load data

This data is produced by the notebook "Rekordbox-Duplicate-Track-Removal-Inator.ipynb"

In [3]:
with open(output_data_directory + '/all_tracks.pickled', 'rb') as f:
    class_all_tracks = pickle.load(f)

In [4]:
with open(output_data_directory + '/tracks_to_remove.pickled', 'rb') as f:
    df_tracks_to_remove = pickle.load(f)

## Define a convenience function that might be reused in the future in other notebooks

In [5]:
def join_by_TrackID(df_data, df_remove):
    df = (
        pd.merge(
            df_data,
            df_remove,
            on = 'TrackID',
            how = 'left'
        )
        .fillna(True)
    )
    return df

## Retrieve dataframe listing tracks to ignore, and filter according to the result

In [6]:
df_only_keep = (
    join_by_TrackID(
        class_all_tracks.df_all_tempo,
        df_tracks_to_remove,
    )
    .fillna(True)
)

df_only_keep = (
    df_only_keep[df_only_keep['keep']]
    .drop(columns = ['keep'])
    .copy()
)

In [7]:
df_only_keep

,TrackID,Inizio,Bpm,Metro,Battito
0,262230753,0.025,128.00,4/4,1.0
1,216118093,0.025,120.00,4/4,1.0
2,216118093,48.026,120.00,4/4,1.0
3,216118093,48.525,120.00,4/4,2.0
4,216118093,49.026,120.00,4/4,3.0
...,...,...,...,...,...
31221,137926098,228.478,127.94,4/4,4.0
31222,137926098,254.272,127.94,4/4,3.0
31223,137926098,281.473,127.94,4/4,1.0
31224,137926098,308.674,127.94,4/4,3.0


## Aggregate to determine the number of tempo changes per track

In [8]:
df_agg = (
    df_only_keep
    .groupby(['TrackID'])
    ['Bpm']
    .agg(['mean', 'sum', 'count'])
    .reset_index()
)

df_agg = (
    pd.merge(
        df_agg,
        class_all_tracks.df_track_info[['TrackID', 'AverageBpm']],
        on = 'TrackID',
        how = 'left',
        
    )
)

In [9]:
df_agg_has_tempo_changes = df_agg[df_agg['count'] > 1]

In [10]:
df_agg_has_tempo_changes

,TrackID,mean,sum,count,AverageBpm
0,156831,77.000000,231.00,3,77.00
1,335159,136.650000,1366.50,10,136.65
5,618560,115.150000,806.05,7,115.15
7,938043,121.600000,3404.80,28,121.60
8,947513,135.000000,270.00,2,135.00
...,...,...,...,...,...
3862,267520197,144.570000,1879.41,13,144.57
3866,267717515,140.140000,980.98,7,140.14
3870,268223065,126.953871,11806.71,93,127.00
3871,268281439,141.390000,3676.14,26,141.38


## Eyeball test

In [11]:
track_id_to_test = 268223065
#track_id_to_test = 268327242
#track_id_to_test = 335159

class_all_tracks.df_all_tempo[class_all_tracks.df_all_tempo['TrackID'] == track_id_to_test]

,TrackID,Inizio,Bpm,Metro,Battito
0,268223065,0.426,126.50,4/4,2.0
1,268223065,18.924,126.56,4/4,1.0
2,268223065,20.820,126.62,4/4,1.0
3,268223065,22.716,126.69,4/4,1.0
4,268223065,24.610,126.75,4/4,1.0
...,...,...,...,...,...
88,268223065,213.630,126.99,4/4,1.0
89,268223065,215.520,126.98,4/4,1.0
90,268223065,217.410,126.99,4/4,1.0
91,268223065,226.860,127.07,4/4,1.0


In [12]:
class_all_tracks.df_track_info[class_all_tracks.df_track_info['TrackID'] == track_id_to_test]

,TrackID,Name,Artist,Composer,Album,Grouping,Genre,Kind,Size,TotalTime,...,BitRate,SampleRate,Comments,PlayCount,Rating,Location,Remixer,Tonality,Label,Mix
879,268223065,Lights On (Nucleus Brown Remix),Wiley (feat. Angel & Tinchy Stryder),,EDM The Collection,,Electronic,MP3 File,9551834,238,...,320,44100,12A - Energy 8,25,0,file://localhost/Users/emily/Desktop/DJ/Tracks...,,12A,,
